##**3 Assignment**

###**다음 미완성 코드를 활용해 새로운 data를 학습해보자**

* Training data image
  * Shape: (?, 3, 64, 128)
    * 여러장의, RGB, 64x128 size의 이미지라고 가정하자

* Test data image
  * Shape: (?, 3, ?, ?)
    * 여러장의, RGB, size를 알 수 없는 이미지라고 가정하자

* Labels
  * image의 class는 2가지 이다
  * data folder의 구성을 참고하자

* Data folder 위치
  * 노트북 github의 data folder 2개를 적당한 위치에 카피해 사용한다

In [17]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import torchvision.datasets as dsets
from torchvision import transforms
import torch.nn as nn
import torch.optim as opt
from torch.utils.data import DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tf = transforms.Compose([transforms.Resize((64,128)), transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])

trainset = dsets.ImageFolder(root = '/content/drive/MyDrive/origin_data',transform=tf)
testset = dsets.ImageFolder(root = '/content/drive/MyDrive/test_data',transform=tf)

trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
testloader = DataLoader(testset, batch_size=16, shuffle=True)

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 32, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2, padding=1))
        self.layer4 = nn.Sequential(nn.Linear(19584, 5000),
                      nn.ReLU(), nn.Dropout()) 
        self.linear1 = nn.Linear(5000, 500)
        self.linear2 = nn.Linear(500, 2) 

    def forward(self,x):
        in_size = x.size(0)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(in_size, -1)
        x = self.layer4(x)
        x = self.linear1(x)
        x = self.linear2(x) 
        return x

learning_rate = 0.005
training_epochs = 10

model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = opt.Adam(model.parameters(),lr=learning_rate)

total_batch = len(trainloader)
for epoch in range(training_epochs):
    total_loss = 0
    for inputs, labels in trainloader:
        inputs,labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss / total_batch

    print('[Epoch: {:>4}] loss = {:>.9}'.format(epoch + 1, avg_loss))

with torch.no_grad():
    for test_inputs, test_labels in testloader:
        test_inputs,test_labels = test_inputs.to(device), test_labels.to(device)
        test_output = model(test_inputs)
        correct_prediction = torch.argmax(test_output, 1) == test_labels
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item() * 100,'%')

#주어진 학습데이터에 대해 모델의 깊이가 깊어 완벽하게 학습되었다. 

[Epoch:    1] loss = 0.0
[Epoch:    2] loss = 0.0
[Epoch:    3] loss = 0.0
[Epoch:    4] loss = 0.0
[Epoch:    5] loss = 0.0
[Epoch:    6] loss = 0.0
[Epoch:    7] loss = 0.0
[Epoch:    8] loss = 0.0
[Epoch:    9] loss = 0.0
[Epoch:   10] loss = 0.0
Accuracy: 100.0 %


In [18]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import torchvision.datasets as dsets
from torchvision import transforms
import torch.nn as nn
import torch.optim as opt
from torch.utils.data import DataLoader
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tf = transforms.Compose([transforms.Resize((64,128)), transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])

trainset = dsets.ImageFolder(root = '/content/drive/MyDrive/origin_data',transform=tf)
testset = dsets.ImageFolder(root = '/content/drive/MyDrive/test_data',transform=tf)

trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
testloader = DataLoader(testset, batch_size=16, shuffle=True)

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 32, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2))
        self.layer2 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2))
        self.layer3 = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1),
                      nn.ReLU(), nn.MaxPool2d(2, stride=2, padding=1))
        self.layer4 = nn.Sequential(nn.Linear(19584, 5000),
                      nn.ReLU(), nn.Dropout()) 
        self.linear1 = nn.Linear(5000, 500)
        self.linear2 = nn.Linear(500, 2) 

    def forward(self,x):
        in_size = x.size(0)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.reshape(in_size, -1)
        x = self.layer4(x)
        x = self.linear1(x)
        x = self.linear2(x) 
        return x

learning_rate = 0.005
training_epochs = 10

model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = opt.Adam(model.parameters(),lr=learning_rate)

for epoch in range(training_epochs):
    loss_sum = 0
    correct1 = 0
    total1 = 0
    for i, data in enumerate(trainloader,0):
        inputs,labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()

        _, pred = torch.max(output.data, 1)
        total1 += labels.size(0)
        correct1 += (pred==labels).sum().item()
        loss_sum += loss.item()
        if i % 2000 == 1999:    
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, loss_sum / 2000))
            loss_sum = 0.0
            
    print('epoch %d Accuracy: %d %%' %(epoch+1, 100*correct1/total1))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device),data[1].to(device)
        output = model(images)
        _, pred1 = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (pred1 == labels).sum().item()

print('Accuracy of test images: %d %%' % (100 * correct / total))

epoch 1 Accuracy: 83 %
epoch 2 Accuracy: 100 %
epoch 3 Accuracy: 97 %
epoch 4 Accuracy: 100 %
epoch 5 Accuracy: 100 %
epoch 6 Accuracy: 100 %
epoch 7 Accuracy: 100 %
epoch 8 Accuracy: 100 %
epoch 9 Accuracy: 100 %
epoch 10 Accuracy: 100 %
Accuracy of test images: 100 %
